In [118]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.utils.multiclass import unique_labels
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD,Adam
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
from keras.utils import to_categorical

In [119]:
(x_train1, y_train), (x_test1, y_test) = tf.keras.datasets.mnist.load_data()
x_train1 = x_train1.reshape(x_train1.shape[0], 28, 28,1)
x_test1 = x_test1.reshape(x_test1.shape[0], 28, 28,1)

In [120]:
x_train = np.empty((60000,32,32,3), dtype=float, order='C')
x_test = np.empty((10000,32,32,3), dtype=float, order='C')
# y_train = np.empty((60000,28,28,3), dtype=float, order='C')
# y_test = np.empty((10000,28,28,3), dtype=float, order='C')

In [121]:
x_test.shape

(10000, 32, 32, 3)

In [122]:
import cv2

In [123]:
x_train1[0].shape

(28, 28, 1)

In [124]:
for i in range(60000):
  x_train[i] = cv2.cvtColor(x_train1[i],cv2.COLOR_GRAY2RGB).resize((32,32,3))
  # x_train[i] = cv2.resize(x_train[i], (32, 32,3))

for i in range(10000):
  x_test[i] = cv2.cvtColor(x_test1[i],cv2.COLOR_GRAY2RGB).resize((32,32,3))
  # x_test[i] = cv2.resize(x_test1[i], (32, 32))

In [125]:
x_test.shape

(10000, 32, 32, 3)

In [126]:
!pip install Keras-Applications

In [127]:
y_train.shape

(60000,)

In [128]:
from keras_efficientnets import EfficientNetB5

In [129]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=.3)

#Dimension of the CIFAR10 dataset
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))


((42000, 32, 32, 3), (42000,))
((18000, 32, 32, 3), (18000,))
((10000, 32, 32, 3), (10000,))


In [130]:
from sklearn.utils.multiclass import unique_labels
from keras.utils import to_categorical

#Since we have 10 classes we should expect the shape[1] of y_train,y_val and y_test to change from 1 to 10
y_train=to_categorical(y_train)
y_val=to_categorical(y_val)
y_test=to_categorical(y_test)

#Verifying the dimension after one hot encoding
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))


((42000, 32, 32, 3), (42000, 10))
((18000, 32, 32, 3), (18000, 10))
((10000, 32, 32, 3), (10000, 10))


In [131]:
from keras_efficientnets import EfficientNetB5

In [132]:
base_model = EfficientNetB5(include_top=False, weights="imagenet", input_shape=(32,32,3),classes=y_train.shape[1])

#Adding the final layers to the above base models where the actual classification is done in the dense layers

model= Sequential()
model.add(base_model) 
model.add(Flatten()) 

#Model summary
# model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
functional_3 (Functional)    (None, 1, 1, 2048)        28513520  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
Total params: 28,513,520
Trainable params: 28,340,784
Non-trainable params: 172,736
_________________________________________________________________


In [133]:
model.add(Dense(1024,activation=('relu'),input_dim=512))

model.add(Dense(512,activation=('relu'))) 
model.add(Dense(256,activation=('relu'))) 
#model.add(Dropout(.3))
model.add(Dense(128,activation=('relu')))
#model.add(Dropout(.2))
model.add(Dense(10,activation=('softmax'))) 

#Checking the final model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
functional_3 (Functional)    (None, 1, 1, 2048)        28513520  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_8 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_9 (Dense)              (None, 10)               

In [134]:
batch_size= 100
epochs=1
learn_rate=.001

In [136]:
sgd=SGD(lr=learn_rate,momentum=.9,nesterov=False)

In [137]:
model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])

#Training the model
model.fit_generator(train_generator.flow(x_train, y_train, batch_size = batch_size), epochs = epochs, steps_per_epoch = x_train.shape[0]//batch_size, validation_data = val_generator.flow(x_val, y_val, batch_size = batch_size), validation_steps = 250,  callbacks = [lrr], verbose = 1)

420/420 [==============================] - 55s 131ms/step - loss: 2.3021 - accuracy: 0.1124 - val_loss: 2.3019 - val_accuracy: 0.1095


In [116]:
scores = model.evaluate(x_test, y_test)

313/313 [==============================] - 7s 22ms/step - loss: 2.3017 - accuracy: 0.1135


In [138]:
scores[0]

2.3017184734344482

In [ ]:
# -*- coding: utf-8 -*-
"""Untitled23.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1KJrUlq1GepTt_dWJErbSBM7j0rLsp5Yo
"""

# -*- coding: utf-8 -*-
"""Untitled20.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/10IhJDTxUdN5ftPedFOvP7PYqXcYPLCJr
"""

from __future__ import print_function
from keras_efficientnets import EfficientNetB5
import keras
import tensorflow as tf 
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from sklearn.utils.multiclass import unique_labels
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os

# Training parameters
def model(parameter,x_train,y_train,x_test,y_test):
  base_model = EfficientNetB5(include_top=False, weights="imagenet", input_shape=(32,32,3),classes=y_train.shape[1])

  #Adding the final layers to the above base models where the actual classification is done in the dense layers
  model= Sequential()
  model.add(base_model) 
  model.add(Flatten()) 

  model.add(Dense(1024,activation=('relu'),input_dim=512))

  model.add(Dense(512,activation=('relu'))) 
  model.add(Dense(256,activation=('relu'))) 
  #model.add(Dropout(.3))
  model.add(Dense(128,activation=('relu')))
  #model.add(Dropout(.2))
  model.add(Dense(10,activation=('softmax'))) 

  #Checking the final model summary
  model.summary()
  batch_size = math.floor(parameter[1])
  epochs = math.floor(parameter[2])
  learn_rate = parameter[0]

  sgd=SGD(lr=learn_rate,momentum=.9,nesterov=False)

  model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])

#Training the model
  model.fit_generator(train_generator.flow(x_train, y_train, batch_size = batch_size), epochs = epochs, steps_per_epoch = x_train.shape[0]//batch_size, validation_data = val_generator.flow(x_val, y_val, batch_size = batch_size), validation_steps = 250,verbose = 1)
  scores = model.evaluate(x_test, y_test)
  return scores[0]





import random
import math
import matplotlib.pyplot as plt
#------------------------------------------------------------------------------
# TO CUSTOMIZE THIS PSO CODE TO SOLVE UNCONSTRAINED OPTIMIZATION PROBLEMS, CHANGE THE PARAMETERS IN THIS SECTION ONLY:
# THE FOLLOWING PARAMETERS MUST BE CHANGED.
def objective_function(x):
    y = 3*(1-x[0])**2*math.exp(-x[0]**2 - (x[1]+1)**2) - 10*(x[0]/5 - x[0]**3 - x[1]**5)*math.exp(-x[0]**2 - x[1]**2) -1/3*math.exp(-(x[0]+1)**2 - x[1]**2);
    return y
 
bounds=[(0.0001,0.1),(16,256),(1,100)]   # upper and lower bounds of variables #learning rate #batch size #epochs #growth rate

nv = 3                   # number of variables
mm = -1                   # if minimization problem, mm = -1; if maximization problem, mm = 1
(x_train1, y_train), (x_test1, y_test) = tf.keras.datasets.mnist.load_data()
x_train1 = x_train1.reshape(x_train1.shape[0], 28, 28,1)
x_test1 = x_test1.reshape(x_test1.shape[0], 28, 28,1)
x_train = np.empty((60000,32,32,3), dtype=float, order='C')
x_test = np.empty((10000,32,32,3), dtype=float, order='C')
for i in range(60000):
  x_train[i] = cv2.cvtColor(x_train1[i],cv2.COLOR_GRAY2RGB).resize((32,32,3))
  # x_train[i] = cv2.resize(x_train[i], (32, 32,3))

for i in range(10000):
  x_test[i] = cv2.cvtColor(x_test1[i],cv2.COLOR_GRAY2RGB).resize((32,32,3))

x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=.3)
y_train=to_categorical(y_train)
y_val=to_categorical(y_val)
y_test=to_categorical(y_test)
# THE FOLLOWING PARAMETERS ARE OPTINAL.
particle_size=1         # number of particles
iterations=1         # max number of iterations
w=0.85                    # inertia constant
c1=1                    # cognative constant
c2=2                     # social constant
# END OF THE CUSTOMIZATION SECTION
#------------------------------------------------------------------------------

class Particle:
    def __init__(self,bounds):
        self.particle_position=[]                     # particle position
        self.particle_velocity=[]                     # particle velocity
        self.local_best_particle_position=[]          # best position of the particle
        self.fitness_local_best_particle_position= initial_fitness  # initial objective function value of the best particle position
        self.fitness_particle_position=initial_fitness             # objective function value of the particle position
 
        for i in range(nv):
            self.particle_position.append(random.uniform(bounds[i][0],bounds[i][1])) # generate random initial position
            self.particle_velocity.append(random.uniform(-1,1)) # generate random initial velocity
 
    def evaluate(self,objective_function):
        self.fitness_particle_position=model(self.particle_position,x_train,y_train,x_test,y_test)
        if mm == -1:
            if self.fitness_particle_position < self.fitness_local_best_particle_position:
                self.local_best_particle_position=self.particle_position                  # update the local best
                self.fitness_local_best_particle_position=self.fitness_particle_position  # update the fitness of the local best
        if mm == 1:
            if self.fitness_particle_position > self.fitness_local_best_particle_position:
                self.local_best_particle_position=self.particle_position                  # update the local best
                self.fitness_local_best_particle_position=self.fitness_particle_position  # update the fitness of the local best
 
    def update_velocity(self,global_best_particle_position):
        for i in range(nv):
            r1=random.random()
            r2=random.random()
 
            cognitive_velocity = c1*r1*(self.local_best_particle_position[i] - self.particle_position[i])
            social_velocity = c2*r2*(global_best_particle_position[i] - self.particle_position[i])
            self.particle_velocity[i] = w*self.particle_velocity[i]+ cognitive_velocity + social_velocity
 
    def update_position(self,bounds):
        for i in range(nv):
            self.particle_position[i]=self.particle_position[i]+self.particle_velocity[i]
 
            # check and repair to satisfy the upper bounds
            if self.particle_position[i]>bounds[i][1]:
                self.particle_position[i]=bounds[i][1]
            # check and repair to satisfy the lower bounds
            if self.particle_position[i] < bounds[i][0]:
                self.particle_position[i]=bounds[i][0]

class PSO():
    def __init__(self,objective_function,bounds,particle_size,iterations):
 
        fitness_global_best_particle_position=initial_fitness
        global_best_particle_position=[]
 
        swarm_particle=[]
        for i in range(particle_size):
            swarm_particle.append(Particle(bounds))
        A=[]
         
        for i in range(iterations):
            print("iterations = ",i)
            for j in range(particle_size):
                swarm_particle[j].evaluate(objective_function)
 
                if mm ==-1:
                    if swarm_particle[j].fitness_particle_position < fitness_global_best_particle_position:
                        global_best_particle_position = list(swarm_particle[j].particle_position)
                        fitness_global_best_particle_position = float(swarm_particle[j].fitness_particle_position)
                if mm ==1:
                    if swarm_particle[j].fitness_particle_position > fitness_global_best_particle_position:
                        global_best_particle_position = list(swarm_particle[j].particle_position)
                        fitness_global_best_particle_position = float(swarm_particle[j].fitness_particle_position)
            for j in range(particle_size):
                swarm_particle[j].update_velocity(global_best_particle_position)
                swarm_particle[j].update_position(bounds)
                 
            A.append(fitness_global_best_particle_position) # record the best fitness
             
             
        print('Optimal solution:', global_best_particle_position)
        print('Objective function value:', fitness_global_best_particle_position)
        print('Evolutionary process of the objective function value:')
        plt.plot(A)
#------------------------------------------------------------------------------
if mm == -1:
    initial_fitness = float("inf") # for minimization problem
if mm == 1:
    initial_fitness = -float("inf") # for maximization problem
#------------------------------------------------------------------------------   
# Main PSO         
# PSO(objective_function,bounds,particle_size,iterations)

PSO(model,bounds,particle_size,iterations)



iterations =  0
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
functional_5 (Functional)    (None, 1, 1, 2048)        28513520  
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dense_11 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_12 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_13 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_14 (Dense)             (None, 10